# "[NeuralNetwork] Convolutional Neural Network"
> KNU AIR week4

- toc: false
- badges: false
- comments: false
- categories: [convolutional neural network]
- hide_{github,colab,binder,deepnote}_badge: true

__Content creators:__ 이중원

__Content reviewers:__ 

# 1. Overview
- Convolutional Neural Network are neural network with __local connectivity__, __shared weights__.
- Conv layer $\rightarrow$ Pooling $\rightarrow$ Conv layer $\rightarrow$ Pooling $\rightarrow$ Flatten $\rightarrow$ Fully connected layer
- Flatten 될 때, translation sensitive 문제 발생

------------------------

# 2. Example

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms

In [8]:
# CUDA
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 

In [9]:
# Data loader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=0, std=1)])

train_dataset = MNIST(root='dataset/', download=True, train=True, transform=transform)
test_dataset = MNIST(root='dataset/', download=True, train=False, transform=transform)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True, num_workers=8)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=True, num_workers=8)

In [10]:
# Model
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.fc(x)
        return x
    
# model
model = CNNModel().to(device)

In [22]:
# optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

# Train

In [23]:
epoch = 10
model.train()

for epoch in range(epoch):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'[EPOCH {epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
    running_loss = 0.0

print('Finished Training')

[EPOCH 1,   469] loss: 0.354
[EPOCH 2,   469] loss: 0.353
[EPOCH 3,   469] loss: 0.353
[EPOCH 4,   469] loss: 0.353
[EPOCH 5,   469] loss: 0.352
[EPOCH 6,   469] loss: 0.351
[EPOCH 7,   469] loss: 0.351
[EPOCH 8,   469] loss: 0.351
[EPOCH 9,   469] loss: 0.352
[EPOCH 10,   469] loss: 0.350
Finished Training


# Test

In [24]:
model.eval()
test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_dataloader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += loss_fn(output, target).item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_dataloader.dataset), 100. * correct / len(test_dataloader.dataset)))


Test set: Average loss: 117.9541, Accuracy: 9691/10000 (97%)

